# Consistent scores

A "consistent scoring function" is a scoring function where following a forecast directive will optimise the forecaster's expected score. It is important to use consistent scoring functions to avoid the "forecaster's dilemma" ([Lerch et al., 2017](https://projecteuclid.org/journals/statistical-science/volume-32/issue-1/Forecasters-Dilemma-Extreme-Events-and-Forecast-Evaluation/10.1214/16-STS588.full)) (i.e., should a forecaster issue an honest forecast or should they issue one that optimises their expected score?). Consistent scoring rules are formally defined in([Gneiting et al., 2011](https://www.tandfonline.com/doi/abs/10.1198/jasa.2011.r10138)).


The MSE is a consistent score for forecasting the mean and the MAE is a consistent score for forecasting the median. But did you know that there are a whole family of scores that are consistent for prediciting the mean, median, or a quantile? 

`score`'s consistent scoring module provides access to these families of consistent scores that can we used to emphasise predictive performance across desired decision thresholds.

### Example 1. A score consistent with the mean
Let's jump into an example that illustrates a scoring function that is consistent with predicting the mean value using `scores`. Suppose we want to evaluate the performance of expected daily precipitation forecasts, but we want to place increasing importance on correctly forecasting extreme values. 


We want to used the `consistent_expectile_score` in `scores`` to evaluate the expected daily precipitation forecasts. Note that medians relate to quantiles in an analogous way to how the mean relates to [expectiles](https://en.wikipedia.org/wiki/Expectile).

Our consistent scoring function for expectiles is 

 $$ S(x, y) =
    \begin{cases}
    (1 - \alpha)(\phi(y) - \phi(x) - \phi'(x)(y-x)), & y < x \\
    \alpha(\phi(y) - \phi(x) - \phi'(x)(y-x)), & x \leq y
    \end{cases}
$$

Where:
- $x$ is the forecast
- $y$ is the observation
- $\alpha$ is the expectile level
- $\phi$ is a [convex function](https://en.wikipedia.org/wiki/Convex_function) of a single variable
- $\phi'$ is the [subderivative](https://en.wikipedia.org/wiki/Subderivative) of $\phi$. The subderivative is a generalisation of the derivative for convex functions and coincides with the derivative when the convex function is differentiable
- $S(x,y)$ is the score.

To make the expectile the consistent with the mean, set $\alpha=0.5$.

Next we need to determine our $\phi(z)$ and $\phi'(z)$ functions. Let's assume that that the importance of predicting extreme rainfall increases exponentially.

First we create the weighting function

$$\phi''(z) = \exp ^\frac{z}{10}$$ 

This will place increasing importance on more extreme rainfall thresholds.

Next we need to integrate $\phi''(z)$ twice so that we can derive the functions for our consistent scoring function.

$$\phi'(z) = 10\exp ^\frac{z}{10}$$ 
and 
$$\phi(z) = z + 100\exp ^\frac{z}{10}$$ 

These equations can be substituted in $ S(x, y)$. Note that to use the `consistent_expectile_score` in scores, you only need to define $\phi(s)$ and $\phi'(s)$, but not $S(x, y)$.

Let's illustrate how this can be done in scores using some synthetic rainfall data.

In [49]:
from scores.continuous import consistent_expectile_score
import numpy as np
import pandas as pd
import xarray as xr

In [78]:
# First define phi and phi prime
def xr_loss_func(s):
    """Phi"""
    result = 0.5 * s**2
    return xr.where(s >=0, result, 0)

def xr_loss_func_prime(s):
    """Phi prime"""
    result = s + (s**3)/6
    return xr.where(s >=0, result, 0)

In [68]:
# Generate some synthetic data
lat = np.linspace(-90, 90, 10)
lon = np.linspace(-180, 180, 20)
times = pd.date_range('2023-11-19', periods=5)

forecast = np.random.uniform(0, 100, size=(len(lat), len(lon), len(times)))
forecast = xr.DataArray(
    forecast,
    dims=('lat', 'lon', 'time'),
    coords={'lat': lat, 'lon': lon, 'time': times}
)

obs = forecast + np.random.random((len(lat), len(lon), len(times)))
obs = obs.clip(min=0)
obs = xr.DataArray(
    obs,
    dims=('lat', 'lon', 'time'),
    coords={'lat': lat, 'lon': lon, 'time': times}
)

In [79]:
# Calculate score
consistent_expectile_score(forecast, obs, alpha=0.5, phi=xr_loss_func, phi_prime=xr_loss_func_prime)

<xarray.DataArray ()>
array(-10152.48286512)

In [82]:
forecast

<xarray.DataArray (lat: 10, lon: 20, time: 5)>
array([[[8.59727480e+01, 5.85597850e+01, 4.90976091e+01, 3.15176177e+01,
         6.29614644e+01],
        [2.39397569e+01, 6.61384984e+01, 9.39778677e+01, 7.34327130e+01,
         5.41211173e+01],
        [5.17437736e+00, 9.51366718e+01, 6.03179266e+01, 7.01687390e+01,
         9.38153054e+01],
        [9.77611523e+01, 7.82267228e+01, 1.95851492e+01, 3.72822967e+01,
         3.58808738e+01],
        [6.28308269e+01, 3.30700809e+01, 3.22562925e+01, 2.08380087e+01,
         8.21707244e+01],
        [4.90971949e+01, 6.33686704e+01, 3.94798553e+01, 5.12649942e+00,
         5.94971340e+01],
        [1.02846131e+01, 8.51916537e+01, 8.67498214e+01, 6.28435665e+00,
         7.42721853e+01],
        [3.99112171e+01, 8.74959833e+01, 8.32438968e+01, 5.68925369e-01,
         9.31489156e+01],
        [6.84436067e+01, 4.01630684e+01, 6.73555466e+01, 1.57053889e+01,
         3.22915543e+01],
        [6.13818056e+01, 7.22829224e+01, 4.86565145e+01, 2.85791274e+01,
         3.81275190e+01],
...
        [1.55615907e+01, 9.88467924e+01, 3.65342080e+01, 3.06835172e+01,
         7.35075841e+01],
        [8.86706225e+01, 1.00228961e+01, 5.96331307e+01, 9.58699032e+01,
         8.23965414e+01],
        [7.12240355e+01, 3.93343182e+01, 4.42500434e+01, 8.73020891e+01,
         2.26180157e+01],
        [3.57183405e+01, 5.63619160e+01, 2.40344101e+01, 9.27762281e+01,
         5.16513825e+01],
        [2.58297953e+01, 1.83078823e+01, 8.80594435e+01, 5.57024349e+01,
         9.23070107e+01],
        [7.31077479e+01, 2.92604580e+01, 7.88147200e+00, 9.11698145e+01,
         6.97020593e+01],
        [3.95411208e+01, 1.81608364e+01, 9.24426266e+01, 8.41531283e+01,
         6.62505564e+01],
        [1.35683629e+01, 5.81301915e+01, 8.24605920e+01, 4.73924812e+01,
         6.88390578e+01],
        [6.88912785e+01, 7.66310638e+01, 5.76383378e+00, 6.14377250e+01,
         7.57467375e+01],
        [8.59322799e+01, 4.61503564e+01, 9.83970576e+01, 6.19109597e+01,
         4.70077862e+01]]])
Coordinates:
  * lat      (lat) float64 -90.0 -70.0 -50.0 -30.0 -10.0 ... 30.0 50.0 70.0 90.0
  * lon      (lon) float64 -180.0 -161.1 -142.1 -123.2 ... 142.1 161.1 180.0
  * time     (time) datetime64[ns] 2023-11-19 2023-11-20 ... 2023-11-23